# 1) Loading the dataset

First things first we load the dataset and gather 1000 sentences like in spacy labs.

In [1]:
import random
from datasets import load_dataset

In [2]:
ds = load_dataset("clarin-knext/fiqa-pl", "corpus")

random.seed(2137)
ds_t1000 = []

for obj in random.sample(list(ds['corpus']),1000):
    ds_t1000.append(obj['text'])

ds_t1000[:3]

['Nie jestem również adwokatem ani doradcą podatkowym. Tak, czynsz, który płacisz swojemu przyjacielowi, jest dochodem podlegającym opodatkowaniu, ale nagle wszelkiego rodzaju wydatki związane z domem – w tym ułamek odsetek zapłaconych od kredytu hipotecznego – stają się odliczane od podatku. Załóżmy na przykład, że kredyt hipoteczny wynosi 1000 zł miesięcznie, a płacisz znajomemu 500 zł miesięcznie. Jeśli mieszkasz w 50% domu, może on odliczyć 50% (plus lub minus) wydatków związanych z posiadaniem domu, w tym: Wszystkie te rzeczy (w każdym razie 50% z nich) podlegają odliczeniu od podatku. Byłoby całkiem możliwe, że poniósłby stratę na przedsięwzięciu i faktycznie co roku obniżał podatki. Dopóki nie nadejdzie czas na sprzedaż; sprzedaż nieruchomości, która była używana jako najem, jest bardziej opodatkowana niż sprzedaż nieruchomości, która była miejscem zamieszkania.',
 '„Jeden scenariusz opisany w pierwotnym pytaniu – niewtajemniczony, który handluje po nieformalnych rozmowach ze zn

# 2) Installing and running ollama in python

In order to use local LLMs, we need to download Ollama. We can do that from [this link](https://ollama.com/download).<br>
After installing Ollama, we need to pull our models. For this lab, I have pulled the following models:
- gemma2 using `ollama pull gemma2`
- phi3:3.8b `ollama pull phi3:3.8b`
Both models have paramteres under 10B

In [3]:
import ollama

Let's see if we can print local models that we downloaded

In [4]:
[print("model: " + model.model, "parameter_size: " + model.details.parameter_size) for model in ollama.list()['models']]

model: gemma2:latest parameter_size: 9.2B
model: phi3:3.8b parameter_size: 3.8B


[None, None]

As we can see, both models have been successfully loaded. For the following tasks, we will exclusively use Gemma-2b to maintain consistency and focus on its specific capabilities.

# 3) Test model query

Let's see how our model will answer a basic question always asked in Poland: `Dokąd nocą tupta jeż?`

In [5]:
# Note: zmienić przy zmianie modelu
model_name = 'gemma2:latest'

In [6]:
from ollama import chat
from ollama import ChatResponse


response: ChatResponse = chat(model=model_name, messages=[
    {
        'role': 'user',
        'content': 'Dokąd nocą tupta jeż?',
    },
])

print(response.message.content)

Nocą jeż tupta **w poszukiwaniu pożywienia**. 

Jeże są głównie nocnymi zwierzętami i wychodzą z swoich schronisk, aby szukać owadów, ślimaków, robaków i innych małych stworzeń. 🦔🐛🐌  



As we can see our local model works correctly and we can proceed with next tasks

# 4) Comparing Entity Recognition: Spacy vs LLM

In this section, we will:
1. Identify all entity categories that Spacy recognizes
2. Ask our LLM to identify entities in the same text
3. Compare the results between Spacy's built-in NER and LLM's entity recognition

The spacy examples will be demonstrated using `displacy` for visual entity representation, alongside the direct response from the LLM.

## a) First example zero-shot prompting

First let's load the spacy model and analyze available entites.

In [7]:
import spacy
from spacy import displacy

nlp = spacy.load('pl_core_news_sm') 
nlp.pipe_labels['ner']

['date', 'geogName', 'orgName', 'persName', 'placeName', 'time']

let's prepare the sentences for spacy, and randomly choose one

In [8]:
docs = list(nlp.pipe(ds_t1000))
docs[3]

"To prawie tak, jakbyś *całkowicie* się mylił. Powiedz mi, co Citibank ma coś wspólnego z JP Morganem. Powiedz mi jeszcze raz, w jaki sposób JP Morgan jest odpowiedzialny za CDO i MBS. Powiedz mi jeszcze raz, jak JP Morgan kosztował podatników. Powiedz mi jeszcze raz, jak banki inwestycyjne dokonywały inwestycji w oparciu o założenie „podatnicy to pokryją”. Powiedz mi jeszcze raz, jak JP Morgan uprawiał hazard i musiał zostać uratowany. Powiedz mi, jak TARP kosztował podatników nawet centa. Zdumiewające, jak ktoś może mieć tak silne uczucia do czegoś, o czym wiedzą. Ucz się debilu.

Spacy's ner has recognize following entites:
- persName: Citibank, JP Morganem
- orgName: JP Morganem, CDO, MBS, TARP 
As we can see spacy did not manage to recognize Citibank as orgName, also it recognize JP Morgan as a persName and orgName.<br><br>
Now let's see the results of gamma2 llm

In [9]:
displacy.render(docs[3], style='ent', page=True)

Now let's define our prompt, we will directly ask model to recognize all entites available in ner

In [10]:
prompt_zero = f"""Odczytaj wszystkie imiona osób, nazwy organizacji,
państw, miejsc, miejsc geograficznych, daty oraz godziny z podanego tekstu. Następnie wypisz znalezione słowa, 
jeżeli wystąpiły w tekście, dzieląc je na odpowiednie kategorie. Oto dany tekst: """

Gamma model properly recognized orgNames as CitiBank and JP Morgan. It did not assign any entites to CDO (Collateralized Debt Obligation), MBS (mortgage-backed securities) and TARP (Troubled Asset Relief Program????)

In [11]:
response: ChatResponse = chat(model=model_name, messages=[
    {
        'role': 'user',
        'content': prompt_zero + ds_t1000[3],
    },
])

print(ds_t1000[3])
print(response.message.content)

"To prawie tak, jakbyś *całkowicie* się mylił. Powiedz mi, co Citibank ma coś wspólnego z JP Morganem. Powiedz mi jeszcze raz, w jaki sposób JP Morgan jest odpowiedzialny za CDO i MBS. Powiedz mi jeszcze raz, jak JP Morgan kosztował podatników. Powiedz mi jeszcze raz, jak banki inwestycyjne dokonywały inwestycji w oparciu o założenie „podatnicy to pokryją”. Powiedz mi jeszcze raz, jak JP Morgan uprawiał hazard i musiał zostać uratowany. Powiedz mi, jak TARP kosztował podatników nawet centa. Zdumiewające, jak ktoś może mieć tak silne uczucia do czegoś, o czym wiedzą. Ucz się debilu.
## Wykryte elementy tekstu:

**Imiona osób:** Nie ma w tekscie nazwisk konkretnych osób.

**Nazwy organizacji:**

* Citibank
* JP Morgan

**Państwa:** Nie ma w tekście nazw państw.

**Miejsca:** Nie ma w tekście nazw miejsc.

**Miejsca geograficzne:** Nie ma w tekście nazw miejsc geograficznych.

**Daty:** Nie ma w tekście dat.

**Godziny:** Nie ma w tekście godzin.


Let me know if you have any other text y

In conclusion the first example showed that llm did a better job in recognizing entities

## b) Running example few-shot prompting

As considering the ner nothig will really change so we need to get ourselves a new example

In [12]:
docs[10]

Wszystkie rozwiązania klimatyczne Zapewnia naprawy i usługi ogrzewania kanałowego w Melbourne. Posiadamy w pełni wykwalifikowany zespół do przeprowadzania dokładnych i spersonalizowanych napraw i serwisowania szerokiej gamy unikalnych systemów grzewczych. Naszym klientom zapewniamy również długotrwałą opiekę posprzedażową.

As we can ner recognize correctly Melborune as a placeName, but also Zapewnia as a persName

In [13]:
displacy.render(docs[10], style='ent', page=True)

In [14]:
prompt_few = f"""Odczytaj wszystkie imiona osób (persName), nazwy organizacji (orgName),
państw (placeName), miejsc (placeName), miejsc geograficznych (geogName), daty (date) oraz godziny (time) z podanego tekstu. Następnie wypisz znalezione słowa, 
jeżeli wystąpiły w tekście, dzieląc je na odpowiednie kategorie. 
Przykładowo w zdaniu. Adam jeździ rano autobusem na zajęcia NLP w Krakowie na uczelni AGH. Możemy znaleźć osobę: Adam, miejsce geograficznę: Kraków, organizację: AGH.
Przykładowo w zdaniu. Ostatnie pokolenie blokowało wisłostradę, wtedy przyjechał Stanowski i ich ośmieszył przed widowanią Kanału ero. Mozemy znaleźć 
osobę: Stanowski, miejsce: wisłostardę, organizację: Ostanie pokolenie, kanału zero
Oto dany tekst: """

Comparing ner with a llm, again Melbourne was corectly recognized as a place this time geoPlace. But again the llm had a one screw-up just like ner. It recognized Wszystkie rozwiązania klimatyczne as a orgName

In [15]:
response: ChatResponse = chat(model=model_name, messages=[
    {
        'role': 'user',
        'content': prompt_few + ds_t1000[10],
    },
])

print(ds_t1000[10])
print(response.message.content)

Wszystkie rozwiązania klimatyczne Zapewnia naprawy i usługi ogrzewania kanałowego w Melbourne. Posiadamy w pełni wykwalifikowany zespół do przeprowadzania dokładnych i spersonalizowanych napraw i serwisowania szerokiej gamy unikalnych systemów grzewczych. Naszym klientom zapewniamy również długotrwałą opiekę posprzedażową.
## Wykryte słowa z tekstu:

**persName:**  Brak

**orgName:** 
* Wszystkie rozwiązania klimatyczne

**placeName:** Melbourne

**placeName:**  Brak

**geogName:** Brak

**date:**  Brak

**time:**  Brak



---


## c) Running more examples both ner and llm.

Preparing sentences. The senteces were chosen randomly of those where ner identified at leadt one entity

In [16]:
indexes = [docs[15], docs[37], docs[34], docs[666], docs[999]]

In [17]:
for sentence in indexes:
  print(sentence)
  print("-------")

:-) Kiepski rząd i ich bezużyteczne normy bezpieczeństwa. Gdyby nie zbyt wiele regulacji, mniej szczęśliwe czteroosobowe rodziny mogłyby teraz jeździć po dziurawej, pełnej dziur autostradzie w przetworzonych puszkach ze spamem i zardzewiałymi igłami bez bezużytecznego ubezpieczenia samochodowego lub przepisów dotyczących emisji.
-------
Czy sprawdziłeś merchresearch.com? Ponadto istnieje kilka dodatków do Chrome, które pokazują BSR (ocena bestsellerów). Nie jestem teraz przy laptopie, żeby ci je dawać, ale Texas Gal Treasures na YouTube zawsze przegląda je w swoich streamach, gdzie projektuje koszulkę od początku do końca. Mam nadzieję że to pomoże!
-------
20 tys. rocznie na uniwersytet stanowy jest o wiele bardziej wiarygodne (pomijajmy „dobre”, ponieważ wtedy musielibyśmy uwzględnić rankingi USNWR, a czesne byłoby zdecydowanie wyższe). Jestem prawie pewien, że średnia czesnego w prywatnej uczelni jest bliższa 40k/rok, może nawet więcej.
-------
Zabrali już twoje pieniądze i siłę nab

Using ner, let's find all entities.

In [18]:
dicts_ner = [{
  "persName": [],
  "orgName": [],
  "placeName": [],
  "geogName": [],
  "date": [],
  "time": []
} for _ in range(len(indexes))]

In [19]:
for i, doc in enumerate(indexes):
  for ent in doc.ents:
    if ent.text not in dicts_ner[i][ent.label_]:    
      dicts_ner[i][ent.label_].append(ent.text)
dicts_ner


[{'persName': ['Kiepski'],
  'orgName': [],
  'placeName': [],
  'geogName': [],
  'date': [],
  'time': []},
 {'persName': ['Chrome'],
  'orgName': ['BSR'],
  'placeName': [],
  'geogName': [],
  'date': [],
  'time': []},
 {'persName': [],
  'orgName': ['USNWR'],
  'placeName': [],
  'geogName': [],
  'date': [],
  'time': []},
 {'persName': [],
  'orgName': ['Fedu'],
  'placeName': ['amerykańskimi'],
  'geogName': [],
  'date': [],
  'time': []},
 {'persName': ['S.', 'C.', 'Wyklucz'],
  'orgName': ['SSN', 'Social Security Administration', 'SS'],
  'placeName': ['Stanach Zjednoczonych', 'USA', 'Stanów Zjednoczonych'],
  'geogName': [],
  'date': [],
  'time': []}]

Using llm

In [20]:
prompt_few = f"""Odczytaj wszystkie imiona osób (persName), nazwy organizacji (orgName),
państw (placeName), miejsc (placeName), miejsc geograficznych (geogName), daty (date) oraz godziny (time) z podanego tekstu. Następnie wypisz znalezione słowa, 
jeżeli wystąpiły w tekście, dzieląc je na odpowiednie kategorie. 
Przykładowo w zdaniu. Adam jeździ rano autobusem na zajęcia NLP w Krakowie na uczelni AGH. Możemy znaleźć osobę: Adam, miejsce geograficznę: Kraków, organizację: AGH.
Przykładowo w zdaniu. Ostatnie pokolenie blokowało wisłostradę, wtedy przyjechał Stanowski i ich ośmieszył przed widowanią Kanału ero. Mozemy znaleźć 
osobę: Stanowski, miejsce: wisłostardę, organizację: Ostanie pokolenie, kanału zero.
Chciałbym też abyś odpowiedzi do pytania zapisywał w taki sposób persName: [(lista znaleziony imion)] (ma to być object w pythonie gdzie kluczem jes właśnie persName).
Zwróć tylko taki objekt bez zbędnych tekstów, nie używaj też nowych lini.
Oto dany tekst: """

In [21]:
prompt_zero = f"""Odczytaj wszystkie imiona osób (persName), nazwy organizacji (orgName),
państw (placeName), miejsc (placeName), miejsc geograficznych (geogName), daty (date) oraz godziny (time) z podanego tekstu. Następnie wypisz znalezione słowa, 
jeżeli wystąpiły w tekście, dzieląc je na odpowiednie kategorie. Chciałbym też abyś odpowiedzi do pytania zapisywał w taki sposób persName: [(lista znaleziony imion)] (ma to być object w pythonie gdzie kluczem jes właśnie persName).
Zwróć tylko taki objekt bez zbędnych tekstów, nie używaj też nowych lini.
Oto dany tekst: """

In [22]:
messages = []
for sentence in indexes:
    message = {
        'role': 'user',
        'content': prompt_few + str(sentence),
    }

    response: ChatResponse = chat(model=model_name, messages=[message])
    messages.append(response)

In [23]:
for msg in messages:
  print(msg.message.content)

persName: []
orgName: ['rząd']
placeName: []
geogName: []
date: []
time: [] 

persName: ['Texas'] 
orgName: ['Chrome', 'YouTube', 'merchresearch.com', 'Texas Gal Treasures']
placeName: ['Texas']
geogName: []
date: []
time: []  

persName: ['Adam', 'Stanowski'],
orgName: ['UNSWWR', 'Ostanie pokolenie', 'Kanału ero'],
placeName: [],
geogName: ['Kraków'],
date: [],
time: [] 

persName: [], 
orgName: ['Fedu', 'unia kredytowa'],
placeName: ['amerykańskimi'],
geogName: [],
date: [],
time: []  

persName: ['Stanowski']
orgName: ['Social Security Administration', 'Ostanie pokolenie', 'Kanału zero'] 
placeName: ['Stany Zjednoczonych', 'USA']
geogName: ['wisłostradę']
date: []
time: [] 







In [24]:
dicts_llm_few = [
    {"persName": [], "orgName": ['rząd'], "placeName": [], "geogName": ['autostradzie'], "date": [], "time": []},
    {"persName": ['Texas'], "orgName": ['Chrome', 'YouTube', 'merchresearch.com', 'Texas Gal Treasures'], 
     "placeName": ['Texas'], "geogName": [], "date": [], "time": []},
    {"persName": [], "orgName": ['USNWR'], "placeName": [], "geogName": [], "date": [], "time": []},
    {"persName": [], "orgName": ['Fedu', 'uni Unii kredytowej'], "placeName": [], "geogName": [], "date": [], "time": []},
    {"persName": [], "orgName": ["Social Security Administration", "Ustawa o zabezpieczeniu społecznym"], 
     "placeName": ["Stany Zjednoczone", "USA"], "geogName": [], "date": [], "time": []}
]

In [25]:
messages_zero = []
for sentence in indexes:
    message = {
        'role': 'user',
        'content': prompt_zero + str(sentence),
    }

    response: ChatResponse = chat(model=model_name, messages=[message])
    messages_zero.append(response)

In [26]:
for msg in messages_zero:
  print(msg.message.content)

persName: [], orgName: [], placeName: [], geogName: [], date: [], time: []  

{'persName': ['Texas Gal'], 'orgName': ['Chrome', 'YouTube', 'merchresearch.com', 'Texas Gal Treasures'], 'placeName': ['Texas'], 'geogName': [], 'date': [], 'time': []} 

{'persName': [], 'orgName': ['USNWR'], 'placeName': [], 'geogName': [], 'date': [], 'time': []}  

{'persName': [], 'orgName': ['Fedu', 'unii kredytowej'], 'placeName': ['amerykańskimi'], 'geogName': [], 'date': [], 'time': []} 

persName: [], orgName: ['Social Security Administration'], placeName: ['Stany Zjednoczone', 'USA'], geogName: [], date: [], time: [] 



In [27]:
dicts_llm_zero = [
  {'persName': [], 'orgName': ['rząd'], 'placeName': [], 'geogName': [], 'date': [], 'time': []},
  {'persName': ['Texas Gal Treasures'], 'orgName': ['Chrome', 'merchresearch.com', 'YouTube'], 'placeName': ['Texas'], 'geogName': [], 'date': [], 'time': []},
  {'persName': [], 'orgName': ['USNWR'], 'placeName': [], 'geogName': [], 'date': [], 'time': []} ,
  {'persName': [], 'orgName': ['Fedu', 'uni'], 'placeName': ['amerykańskie'], 'geogName': [], 'date': [], 'time': []},
  {"persName": [],"orgName": ["Social Security Administration"]," placeName": ["Stany Zjednoczone", "USA"], "geogName": [],"date": [],"time": []} 
  
]

Comparaing results from ner and llm

In [28]:
for i, (ner, llm_f, llm_z) in enumerate(zip(dicts_ner, dicts_llm_few, dicts_llm_zero)):
    print("Sentence: ", indexes[i])
    print("Ner: ", ner)
    print("LLM few-shot: ", llm_f)
    print("LLM zero-shot: ", llm_z)
    print("--------------")


Sentence:  :-) Kiepski rząd i ich bezużyteczne normy bezpieczeństwa. Gdyby nie zbyt wiele regulacji, mniej szczęśliwe czteroosobowe rodziny mogłyby teraz jeździć po dziurawej, pełnej dziur autostradzie w przetworzonych puszkach ze spamem i zardzewiałymi igłami bez bezużytecznego ubezpieczenia samochodowego lub przepisów dotyczących emisji.
Ner:  {'persName': ['Kiepski'], 'orgName': [], 'placeName': [], 'geogName': [], 'date': [], 'time': []}
LLM few-shot:  {'persName': [], 'orgName': ['rząd'], 'placeName': [], 'geogName': ['autostradzie'], 'date': [], 'time': []}
LLM zero-shot:  {'persName': [], 'orgName': ['rząd'], 'placeName': [], 'geogName': [], 'date': [], 'time': []}
--------------
Sentence:  Czy sprawdziłeś merchresearch.com? Ponadto istnieje kilka dodatków do Chrome, które pokazują BSR (ocena bestsellerów). Nie jestem teraz przy laptopie, żeby ci je dawać, ale Texas Gal Treasures na YouTube zawsze przegląda je w swoich streamach, gdzie projektuje koszulkę od początku do końca. M

As we can see there was only one sentence where both llm prompts and ner got the same result (3rd sentence). Comparing all results we can see that Llm with a few-shot prompt gather more correct information. Although looking at the last example (5th sentence) the zero-shot version corectly did not put `Ustawa (...)` as orgName. Overall ner classifies much more information but uncorectly. <br>
Funny thing in 2nd sentence ner classified `Chrome` as persName while LLM zero classifed `Texas Gal Treasures` as name but also classifed `Texas` as placeName.
Searching through web `Texas` is indeed a person name "The name Texas is primarily a gender-neutral name of Native American - Caddo origin that means Friend." 
[source link of the name](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwjfrY_amqiKAxVnJRAIHfAhBHUQFnoECBAQAw&url=https%3A%2F%2Fbabynames.com%2Fname%2Ftexas&usg=AOvVaw0nCGZu-cmPCFjKU32rP72X&opi=89978449)

# 5) Building a simple evaluation pipeline 

## Metrics for NER

First let's calculate the metrics `recall`,`f1-score`,`precision` for NER method 

Let's load our dataset, prepared by students.

In [58]:
import pandas as pd

csv_file_path = '../data/lab7_input.csv'  
df = pd.read_csv(csv_file_path, sep=';')
df.head()

,Text,Entities
0,"Niemal dwa tygodnie temu doszło do walki, na k...","[('Jake Paul', 'persName', (119, 128)), (""Mike..."
1,Te problemy widać już na początku. Ridley Scot...,"[('Ridley Scott', 'persName', (35, 47)), ('Aca..."
2,Phoenix Suns po dobrym starcie sezonu ostatnio...,"[('Phoenix Suns', 'orgName', (0, 12)), ('Arizo..."
3,Oświadczenie Karola Nawrockiego: Szef Instytut...,"[('Karola Nawrockiego', 'persName', (13, 31)),..."
4,Brandin Podziemski występuje w Golden State Wa...,"[('Brandin Podziemski', 'persName', (0, 18)), ..."


As we can see the Entities column is semi designed for Scorer from spacy, let's see one record. As we can see We have 3 values in Entiteis the word, entity and place in string.

In [108]:
print(df.iloc[-3]['Entities'])
print(df.iloc[-3]['Text'])

[("Jan Paweł 2","persName",(0,11)),("Polski","placeName",(39,45)),("Watykanie","placeName",(61,70))]
Jan Paweł 2 był papieżem pochodzącym z Polski, urzędujacym w Watykanie.


Now we need to create a new entities list for spacy Scorer, every item should have a begining (where word starts), end (where word ends), entity.

In [62]:
import ast

entities = []

for index, row in df.iterrows():
  entities_list = ast.literal_eval(row['Entities'])
  entities.append([(ent[2][0], ent[2][1], ent[1]) for ent in entities_list])
  
entities[0]

[(119, 128, 'persName'),
 (137, 150, 'persName'),
 (297, 315, 'persName'),
 (83, 95, 'orgName'),
 (98, 107, 'placeName'),
 (110, 118, 'placeName')]

Let's run the Scorer and calculate metrics

In [75]:
from spacy.training import Example
from spacy.scorer import Scorer


examples = []

def evaluate(ner_model, df, entities):
    
    scorer = Scorer()

    for (index, row), annot in zip(df.iterrows(), entities):
        text = row['Text']
        doc = ner_model.make_doc(text)
        example = Example.from_dict(doc, {"entities": annot})
        example.predicted = nlp(str(example.predicted))
        examples.append(example)
        #print(spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), annot))
    score = scorer.score(examples)
    return score

In [77]:
score = evaluate(nlp, df, entities)

d:\Studia\NLP\.venv\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Własny komitet i rozdawanie pieniędzy: Większość ś..." with entities "[(86, 91, 'persName'), (101, 112, 'orgName'), (161...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
d:\Studia\NLP\.venv\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Zwycięstwo Donalda Trumpa w wyborach prezydenckich..." with entities "[(11, 25, 'persName'), (53, 56, 'placeName'), (130...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
d:\Studia\NLP\.venv\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligne

As we can see below the average precision of all values is not that great 0.48. Looking at every Entity, we can see the reason date or time have really low values there. Looking at Recall we can see that it's value is highre than precision which means ner tends to over-identify entities. Looking through every entitiy we can see that placeName had the best performance.

In [110]:
precision = score["ents_p"]
recall = score["ents_r"]
f1_score = score["ents_f"]

print(f"Overall Precision: {round(precision, 2) * 100}%")
print(f"Overall Recall: {round(recall, 2)* 100}%")
print(f"Overall F1 Score: {round(f1_score,2)* 100}%")

for entity, metrics in score["ents_per_type"].items():
    print(f"Entity Type: {entity}")
    print(f"  Precision: {round(metrics['p'],2)* 100}%")
    print(f"  Recall: {round(metrics['r'],2)* 100}%")
    print(f"  F1 Score: {round(metrics['f'],2)* 100}%")

Overall Precision: 48.0%
Overall Recall: 56.99999999999999%
Overall F1 Score: 52.0%
Entity Type: placeName
  Precision: 54.0%
  Recall: 73.0%
  F1 Score: 62.0%
Entity Type: geogName
  Precision: 15.0%
  Recall: 40.0%
  F1 Score: 22.0%
Entity Type: persName
  Precision: 46.0%
  Recall: 67.0%
  F1 Score: 54.0%
Entity Type: orgName
  Precision: 60.0%
  Recall: 43.0%
  F1 Score: 50.0%
Entity Type: date
  Precision: 17.0%
  Recall: 28.999999999999996%
  F1 Score: 21.0%
Entity Type: time
  Precision: 0.0%
  Recall: 0.0%
  F1 Score: 0.0%


## Metrics for LLM

We can use the same prompt

In [80]:
prompt_few_score = f"""Odczytaj wszystkie imiona osób (persName), nazwy organizacji (orgName),
państw (placeName), miejsc (placeName), miejsc geograficznych (geogName), daty (date) oraz godziny (time) z podanego tekstu. Następnie wypisz znalezione słowa, 
jeżeli wystąpiły w tekście, dzieląc je na odpowiednie kategorie. 
Przykładowo w zdaniu. Adam jeździ rano autobusem na zajęcia NLP w Krakowie na uczelni AGH. Możemy znaleźć osobę: Adam, miejsce geograficznę: Kraków, organizację: AGH.
Przykładowo w zdaniu. Ostatnie pokolenie blokowało wisłostradę, wtedy przyjechał Stanowski i ich ośmieszył przed widowanią Kanału ero. Mozemy znaleźć 
osobę: Stanowski, miejsce: wisłostardę, organizację: Ostanie pokolenie, kanału zero.
Chciałbym też abyś odpowiedzi do pytania zapisywał w taki sposób persName: [(lista znaleziony imion)] (ma to być object w pythonie gdzie kluczem jes właśnie persName).
Zwróć tylko taki objekt bez zbędnych tekstów, nie używaj też nowych lini.
Oto dany tekst: """

In [83]:
messages_score = []
for index, sentence in df.iterrows():
    message = {
        'role': 'user',
        'content': prompt_few_score + str(sentence),
    }

    response: ChatResponse = chat(model=model_name, messages=[message])
    messages_score.append(response)

Let's save the output to file, although llm returns data as python objects sometimes it adds additional text so I need to fix that by hand

In [89]:
with open('../data/lab7_llm_output.txt', "w", encoding="utf-8") as file:
    file.writelines([line.message.content + "\n" for line in messages_score])

In [91]:
path_ = '../data/lab7_llm_output.txt'

llm_outputs = []
with open(path_, 'r', encoding='utf-8') as file:
    for line in file:
        if line.strip():
            try:
                obj = eval(line.strip())
                llm_outputs.append(obj)
            except:
                continue
llm_outputs[-3]

{'persName': ['Jan Paweł 2']}

Let's prepare the the True values in list of python objects

In [103]:
entities_llm = [{
  "persName": [],
  "orgName": [],
  "placeName": [],
  "geogName": [],
  "date": [],
  "time": []
} for _ in range(len(entities))]

for index, row in df.iterrows():
  entities_list = ast.literal_eval(row['Entities'])
  for ent in entities_list:
    entities_llm[index][ent[1]].append(ent[0])
  
entities_llm[-3]

{'persName': ['Jan Paweł 2'],
 'orgName': [],
 'placeName': ['Polski', 'Watykanie'],
 'geogName': [],
 'date': [],
 'time': []}

In [104]:
result_obj = {
  'persName': {
    'True': 0,
    'False': 0
  },
  'orgName': {
    'True': 0,
    'False': 0
  },
  'geogName': {
    'True': 0,
    'False': 0
  },
  'date': {
    'True': 0,
    'False': 0
  },
  'time': {
    'True': 0,
    'False': 0
  },
}

Let's calculate the metrics for llm. As we can see the llm result are much better than ner. Overall precision sits at 77.69% while ner's was 48.0%. The llm method did much more better job than ner especially in time and date field. The llm is conservative in its predictions - when it identifies entities, it's usually correct, but misses many entities

In [107]:
total_tp = 0
total_fp = 0
total_fn = 0

metrics = {'overall': {}, 'entities': {}}

for entity_type, counts in result_obj.items():
    true_positives = counts['True']
    false_negatives = counts['False']
    false_positives = 0
    
    for pred in llm_outputs:
        pred_entities = set(pred.get(entity_type, []))
        gold_entities = set(entities_llm[llm_outputs.index(pred)][entity_type])
        false_positives += len(pred_entities - gold_entities)

    total_tp += true_positives
    total_fp += false_positives
    total_fn += false_negatives

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    metrics['entities'][entity_type] = {
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

    print(f"\n{entity_type}:")
    print(f"Precision: {precision:.2%}")
    print(f"Recall: {recall:.2%}")
    print(f"F1 Score: {f1:.2%}")

overall_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
overall_recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
overall_f1 = 2 * (overall_precision * overall_recall) / (overall_precision + overall_recall) if (overall_precision + overall_recall) > 0 else 0

metrics['overall'] = {
    'precision': overall_precision,
    'recall': overall_recall,
    'f1': overall_f1
}

print("\nOVERALL METRICS:")
print(f"Precision: {overall_precision:.2%}")
print(f"Recall: {overall_recall:.2%}")
print(f"F1 Score: {overall_f1:.2%}")


persName:
Precision: 88.89%
Recall: 40.00%
F1 Score: 55.17%

orgName:
Precision: 62.50%
Recall: 19.23%
F1 Score: 29.41%

geogName:
Precision: 80.00%
Recall: 28.57%
F1 Score: 42.11%

date:
Precision: 83.33%
Recall: 35.71%
F1 Score: 50.00%

time:
Precision: 100.00%
Recall: 100.00%
F1 Score: 100.00%

OVERALL METRICS:
Precision: 77.69%
Recall: 29.38%
F1 Score: 42.63%


# 6) Answering questions

#### How does the performance of LLM-based NER compare to traditional approaches? What are the trade-offs in terms of accuracy, speed, and resource usage?

The llm method was more precise but also it took 10 minutes to calculate all results. Also ner model is not nearly as big as llm to store. NER results were much more worse than LLM's when we made our tests. 

#### Which prompting strategy proved most effective for NER and classification tasks? Why?

The few-shot method proved itself being better than zero method. I think that a few examples provided enough to make it recognize more companies or people names. In conlusion to prompts the bigger the better.

#### What are the limitations and potential biases of using LLMs for NER and classification?

The limitations definitely include the significant computational resources required to process all the messages sent to an LLM. Regarding potential biases, the training data used to develop an LLM can be problematic if it does not include names of people from diverse countries or geographical regions.

#### In what scenarios would you recommend using traditional NER vs. LLM-based approaches?

NER:
- very usefull for big data does not need as much resources as llm
- Working with one language
- When entities are well defined

LLM:
- Dealing with complex entities
- When acccuracy is more important than speed
- Working with words from diffrent languages, for example polish sentence using english words=